In [50]:
#! pip install mysql-connector-python
#! pip install PyMySQL
import json
import time
import urllib.request
import os
import pprint
import requests
import requests.exceptions
import pandas as pd
import numpy
from sqlalchemy import create_engine
import mysql.connector
import matplotlib.pyplot as plt
import pymysql
from mysql.connector import Error

In [51]:
regionCode=input()

 EUR


In [52]:
apikey="8uZr0xabEcXEvyShuxanmDjzIemBghza"
url="http://dataservice.accuweather.com/locations/v1/countries/"+regionCode+"?apikey=8uZr0xabEcXEvyShuxanmDjzIemBghza"
querystring={"Region Code":regionCode}
headers={
    'x-api-key':apikey
    }
response=requests.request("GET",url,headers=headers,params=querystring)
print(response.text)
dict=response.text

# API Error Handling
if dict=="[]":
    print("This Region Code does not exist.")

[{"ID":"AD","LocalizedName":"Andorra","EnglishName":"Andorra"},{"ID":"AL","LocalizedName":"Albania","EnglishName":"Albania"},{"ID":"AT","LocalizedName":"Austria","EnglishName":"Austria"},{"ID":"BA","LocalizedName":"Bosnia and Herzegovina","EnglishName":"Bosnia and Herzegovina"},{"ID":"BE","LocalizedName":"Belgium","EnglishName":"Belgium"},{"ID":"BG","LocalizedName":"Bulgaria","EnglishName":"Bulgaria"},{"ID":"BY","LocalizedName":"Belarus","EnglishName":"Belarus"},{"ID":"CH","LocalizedName":"Switzerland","EnglishName":"Switzerland"},{"ID":"CY","LocalizedName":"Cyprus","EnglishName":"Cyprus"},{"ID":"CZ","LocalizedName":"Czechia","EnglishName":"Czechia"},{"ID":"DE","LocalizedName":"Germany","EnglishName":"Germany"},{"ID":"DK","LocalizedName":"Denmark","EnglishName":"Denmark"},{"ID":"EE","LocalizedName":"Estonia","EnglishName":"Estonia"},{"ID":"ES","LocalizedName":"Spain","EnglishName":"Spain"},{"ID":"FI","LocalizedName":"Finland","EnglishName":"Finland"},{"ID":"FO","LocalizedName":"Faroe I

In [53]:
df=pd.DataFrame(eval(dict))
df

,ID,LocalizedName,EnglishName
0,AD,Andorra,Andorra
1,AL,Albania,Albania
2,AT,Austria,Austria
3,BA,Bosnia and Herzegovina,Bosnia and Herzegovina
4,BE,Belgium,Belgium
5,BG,Bulgaria,Bulgaria
6,BY,Belarus,Belarus
7,CH,Switzerland,Switzerland
8,CY,Cyprus,Cyprus
9,CZ,Czechia,Czechia


In [ ]:
# Saved these API results as a CSV file to input as a dimension table for my SQL database (project_dw)
df.to_csv('countryapi.csv')

In [54]:
host_name = "localhost"
port = "3306"
user_id = "ds2002"
pwd = "Uva!1819"

db_name = "project_dw"

In [55]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    connection = None
   
    try:
        conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
        sqlEngine = create_engine(conn_str, pool_recycle=3600)
        connection = sqlEngine.connect()
        dframe = pd.read_sql(sql_query, connection);
    
    except Error:
        print("Connection not working. Check if SQL database exists.")
    
    connection.close()
    return dframe
    


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [56]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
from sqlalchemy import text

In [75]:
# Select COUNTRY dimension table from project_dw --> first dimension table
# Data Source 1 from the World SQL database
sql_country = "SELECT * FROM project_dw.dim_country;"
df_country = get_dataframe(user_id, pwd, host_name, db_name, text(sql_country))
df_country.head(5)

,Code,Name,Continent,Region,SurfaceArea,Population,LifeExpectancy,GNP,GovernmentForm,HeadOfState
0,ABW,Aruba,North America,Caribbean,193.0,103000,78.4,828.0,Nonmetropolitan Territory of The Netherlands,Beatrix
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,22720000,45.9,5976.0,Islamic Emirate,Mohammad Omar
2,AGO,Angola,Africa,Central Africa,1246700.0,12878000,38.3,6648.0,Republic,José Eduardo dos Santos
3,AIA,Anguilla,North America,Caribbean,96.0,8000,76.1,63.2,Dependent Territory of the UK,Elisabeth II
4,ALB,Albania,Europe,Southern Europe,28748.0,3401200,71.6,3205.0,Republic,Rexhep Mejdani


In [77]:
# Delete unnecessary columns from COUNTRY dimension
sql_newcountry = "ALTER TABLE dim_country DROP COLUMN GovernmentForm, DROP COLUMN HeadOfState;"

df_country = get_dataframe(user_id, pwd, host_name, db_name, text(sql_country))
df_country.head(5)

,Code,Name,Continent,Region,SurfaceArea,Population,LifeExpectancy,GNP
0,ABW,Aruba,North America,Caribbean,193.0,103000,78.4,828.0
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,22720000,45.9,5976.0
2,AGO,Angola,Africa,Central Africa,1246700.0,12878000,38.3,6648.0
3,AIA,Anguilla,North America,Caribbean,96.0,8000,76.1,63.2
4,ALB,Albania,Europe,Southern Europe,28748.0,3401200,71.6,3205.0


In [59]:
# Select COUNTRYAPI dimension table from project_dw --> second dimension table
# Data Source 2 from above API Call
sql_countryapi = "SELECT * FROM project_dw.dim_countryapi;"
df_countryapi = get_dataframe(user_id, pwd, host_name, db_name, text(sql_countryapi))
df_countryapi.head(5)

,ID,LocalizedName,EnglishName
0,AD,Andorra,Andorra
1,AL,Albania,Albania
2,AT,Austria,Austria
3,BA,Bosnia and Herzegovina,Bosnia and Herzegovina
4,BE,Belgium,Belgium


In [60]:
# Select TOURISM fact table from project_dw --> fact table
# Data Source 3 from CSV file, whih is all about tourism, was loaded into SQL 
sql_tourismm = "SELECT * FROM project_dw.fact_tourismm;"
df_tourismm = get_dataframe(user_id, pwd, host_name, db_name, text(sql_tourismm))
df_tourismm.head(5)

,Ranking,Country,Tourists,Year,Region,Population,ID,Code
0,26,Albania,2658000,2020,Southern Europe,3401200,AL,ALB
1,18,Andorra,5207000,2020,Southern Europe,78000,AD,AND
2,11,Austria,15091000,2020,Western Europe,8091800,AT,AUT
3,27,Belgium,2584000,2020,Western Europe,10239000,BE,BEL
4,20,Bulgaria,4973000,2020,Eastern Europe,8190900,BG,BGR


In [61]:
# AVG number of tourists in Europe
sql_touravg = "SELECT round(AVG(Tourists)) as 'Average # of Tourists' FROM fact_tourismm;"
df_touravg = get_dataframe(user_id, pwd, host_name, db_name, text(sql_touravg))
df_touravg

,Average # of Tourists
0,12982589.0


In [62]:
# SUM of tourists in Europe in 2020
sql_tourin2020 = "SELECT SUM(Tourists) as 'Total Tourists in 2020' FROM fact_tourismm WHERE Year=2020;"
df_tourin2020 = get_dataframe(user_id, pwd, host_name, db_name, text(sql_tourin2020))
df_tourin2020

,Total Tourists in 2020
0,376033400.0


In [63]:
# SUM of tourists from the top 5 ranking countries for tourism rates
sql_tourtop5 = "SELECT SUM(Tourists) as 'Total Tourists' FROM fact_tourismm WHERE Ranking<=5;"
df_tourtop5 = get_dataframe(user_id, pwd, host_name, db_name, text(sql_tourtop5))
df_tourtop5

,Total Tourists
0,280453000.0


In [64]:
# COUNT countries per region
sql_region = "SELECT COUNT(Country) as 'Number of Countries', region as 'Region' FROM fact_tourismm GROUP BY region;"
df_region = get_dataframe(user_id, pwd, host_name, db_name, text(sql_region))
df_region

,Number of Countries,Region
0,11,Southern Europe
1,9,Western Europe
2,7,Eastern Europe
3,2,Middle East
4,4,Nordic Countries
5,3,Baltic Countries
6,2,British Islands


In [65]:
# Select Ranking and Country and ORDER BY Population
# Offers insight into if tourism ranking is associated with country population
# Do more people visit countries with a larger population?
sql_pop = "SELECT Ranking, Country, Population FROM fact_tourismm ORDER BY Population DESC;"
df_pop = get_dataframe(user_id, pwd, host_name, db_name, text(sql_pop))
df_pop

,Ranking,Country,Population
0,12,Germany,82164700
1,8,Turkey,66591000
2,14,United Kingdom,59623400
3,1,France,59225700
4,3,Italy,57680000
5,24,Ukraine,50456000
6,5,Spain,39441700
7,2,Poland,38653600
8,19,Romania,22455500
9,17,Netherlands,15864000


In [66]:
# AVG number of tourists per region
sql_tpr = "SELECT round(AVG(Tourists)) as 'Avg. Number of Tourists', region as 'Region' FROM fact_tourismm GROUP BY region;"
df_tpr = get_dataframe(user_id, pwd, host_name, db_name, text(sql_tpr))
df_tpr

,Avg. Number of Tourists,Region
0,10904636.0,Southern Europe
1,18562044.0,Western Europe
2,19594429.0,Eastern Europe
3,10044000.0,Middle East
4,4961250.0,Nordic Countries
5,2394333.0,Baltic Countries
6,11026000.0,British Islands
